In [ ]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [ ]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 3. Building an Environment

In [113]:
# Define the custom environment
class ShowerEnv(Env):
    def __init__(self):
        self.N=3
        self.path = []  # Start with the initial state
        self.state = 0
        self.action_space = Discrete(9)
        self.observation_space = Discrete(9)
        self.reward=[]
        self.episode = 100
        self.reward_matrix = np.array([
            [-100, 10, 0, 0, 0, 0, 0, 0, 0],
            [-20, -100, -50, 0, 10, 0, 0, 0, 0],
            [0, 0, -100, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, -100, 0, 0, 0, 0, 0],
            [0, -20, 0, -50, -100, -50, 0, 10, 0],
            [0, 0, 0, 0, 0, -100, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, -100, -20, 0],
            [0, 0, 0, 0, -20, 0, 150, -100, 100],
            [0, 0, 0, 0, 0, 0, 0, 0, -150],
        ])
        

    def newaction(self):
        x=self.state
        samples=[x+self.N,x-self.N]

        if (x+1)%self.N != 0 :
            samples.append(x+1)
        if (x-1)%self.N != self.N - 1 :
            samples.append(x-1)

        samples=[sample for sample in samples if sample in range(0,10)]   
        # print(x,samples)
        newaction=random.choice(samples)
        return newaction
    



    def step(self, action):
        
        reward = self.reward_matrix[self.state, action]
    
        # Rule 1: If agent encounters -50, game terminates
        if reward == -50:
            self.state = action
            done = True
    # Rule 3: Check for self-looping
        elif self.state == action:
            reward = -100
            done = False
    # Rule 2: Check for no direct edge
        elif reward == 0:
            done = False
    # Rule 4: All other cases, transition and follow reward
        else:
            self.state = action
            done = False
    
                
        
        if self.episode <= 0 or reward==100:
            done = True

        self.episode -= 1


        info = {}
         # Tracking path without repetitions
        if not self.path or self.state != self.path[-1]:  # Check if path is empty or the current state is different from the last state in the path
            self.path.append(self.state)
            self.reward.append(reward)
        
        

        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        print("Path taken:", self.path)  # Print the path taken
        self.path = [0]  # Reset the path for the next episode
        self.state = random.randint(0,8)
        self.episode = 100
        print("reward--->",self.reward)
        self.reward=[]
        return self.state



In [114]:
env=ShowerEnv()

# 4. Test Environment

In [116]:
episodes = 5

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    # states=[]
    
    while not done:
        env.render()
        action = env.newaction()
        # print("old state, action ", action)
        
        n_state, reward, done, info= env.step(action)
        print("reward afetr action--->",reward)
        score+=reward
        # states.append(state)
        # states.append(n_state)
        
    print('Episode:{} Score:{}'.format(episode, score))
    print("---------------------------------------------------------------")

env.close()

Path taken: [0]
reward---> []
reward afetr action---> -20
reward afetr action---> -50
Episode:1 Score:-70
---------------------------------------------------------------
Path taken: [0, 4, 3]
reward---> [-20, -50]
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward afetr action---> 0
reward

In [112]:
env.close()

# 5. Train Model

In [100]:
log_path = os.path.join('Training', 'Logs')

In [101]:
model = DQN("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/bidyut/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [102]:
model.learn(total_timesteps=80000)

Path taken: [0, 1, 2]
reward---> [10, -50]
Path taken: [0, 2]
reward---> [0]
Path taken: [0, 2]
reward---> [0]
Path taken: [0, 1, 0, 1, 0, 1, 0, 1, 2]
reward---> [10, -20, 10, -20, 10, -20, 10, -50]
Path taken: [0, 1, 2]
reward---> [-100, -50]
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 66.8     |
|    ep_rew_mean      | -710     |
|    exploration_rate | 0.968    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7870     |
|    time_elapsed     | 0        |
|    total_timesteps  | 267      |
----------------------------------
Path taken: [0, 1, 4, 3]
reward---> [10, 10, -50]
Path taken: [0, 1, 0, 1, 2]
reward---> [0, -20, 10, -50]
Path taken: [0, 1, 2]
reward---> [10, -50]
Path taken: [0, 1, 2]
reward---> [10, -50]
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 41.9     |
|    ep_rew_mean      | -526     |
|    exploration_rate | 0.96     |
| t

# 6. Save Model

In [103]:
model.save('DQN_s')

In [104]:
model = DQN.load('DQN_s', env)

evaluate_policy(model, env, n_eval_episodes=4, render=True)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Path taken: [0, 3]
reward---> [0]
Path taken: [0, 2]
reward---> [0]
Path taken: [0, 2]
reward---> [0]
Path taken: [0, 4, 7, 8]
reward---> [10, 10, 100]
Path taken: [0, 2]
reward---> [0]


/home/bidyut/.local/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/home/bidyut/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(30.0, 51.96152422706632)

In [108]:
obs = env.reset()
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

Path taken: [0]
reward---> []


In [79]:
reward

10